# A2C (Advantage Actor-Critic) - Interactive Exercise

Welcome! In this notebook, you will implement **A2C (Advantage Actor-Critic)**, an improved version of the basic Actor-Critic algorithm.

## What is A2C?

A2C is the **synchronous** version of A3C (Asynchronous Advantage Actor-Critic). It improves upon basic Actor-Critic by:
- Using **n-step returns** instead of 1-step TD
- Explicitly computing **advantages** (not just TD error)
- Better **bias-variance tradeoff**

## Key Differences from Actor-Critic

| Aspect | Actor-Critic (1-step) | A2C |
|--------|----------------------|-----|
| Returns | 1-step TD: r + γV(s') | **n-step**: r₁ + γr₂ + ... + γⁿV(s_{t+n}) |
| Advantage | TD error: δ = r + γV(s') - V(s) | **n-step advantage**: A(s,a) = R_n - V(s) |
| Bias-Variance | Higher bias, lower variance | **Better tradeoff** |
| Sample Efficiency | Lower | **Higher** |
| Stability | Good | **Better** |

## N-Step Returns

The key innovation is using **n-step returns**:

$$R_t^{(n)} = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + ... + \gamma^{n-1} r_{t+n-1} + \gamma^n V(s_{t+n})$$

This provides a better estimate than 1-step TD by looking further ahead!

## Learning Objectives

By the end of this notebook, you will:
- Understand n-step returns and why they help
- Implement n-step advantage calculation
- Build a proper A2C agent
- Compare A2C with basic Actor-Critic
- See the bias-variance tradeoff in action

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gymnasium as gym
import matplotlib.pyplot as plt
from collections import deque
from a2c_tests import *

In [ ]:
# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

## The Environment: CartPole

We'll use CartPole-v1 to compare A2C with basic Actor-Critic.

- **State**: [position, velocity, angle, angular velocity]
- **Actions**: 0 (left), 1 (right)
- **Reward**: +1 per timestep
- **Success**: Average reward > 475

In [ ]:
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

print(f"State dimension: {state_dim}")
print(f"Action dimension: {action_dim}")

## Exercise 1: Compute N-Step Returns

The core of A2C is computing **n-step returns**. Given a trajectory of experiences, we need to compute the discounted sum of rewards looking n steps ahead.

**Formula**:
$$R_t^{(n)} = \sum_{i=0}^{n-1} \gamma^i r_{t+i} + \gamma^n V(s_{t+n})$$

If the episode ends before n steps, we only sum up to the terminal state.

**Task**: Implement n-step returns calculation.

In [ ]:
# GRADED FUNCTION: compute_n_step_returns

def compute_n_step_returns(rewards, values, dones, gamma=0.99, n_steps=5):
    """
    Compute n-step returns for a batch of experiences.
    
    Arguments:
    rewards -- list of rewards [r_0, r_1, ..., r_T]
    values -- list of state values [V(s_0), V(s_1), ..., V(s_T), V(s_{T+1})]
              Note: values has one more element (bootstrap value)
    dones -- list of done flags [d_0, d_1, ..., d_T]
    gamma -- discount factor
    n_steps -- number of steps to look ahead
    
    Returns:
    returns -- list of n-step returns [R_0, R_1, ..., R_T]
    """
    # (approx. 15-18 lines)
    # 1. Initialize returns list
    # 2. For each timestep t:
    #    a. Initialize n_step_return = 0
    #    b. For i in range(n_steps):
    #       - Check if t+i is within bounds
    #       - If done[t+i] is True:
    #         * Add gamma^i * reward[t+i]
    #         * Break (episode ended)
    #       - Else:
    #         * Add gamma^i * reward[t+i]
    #       - If reached n_steps and not done:
    #         * Add gamma^n * V(s_{t+n}) as bootstrap
    #    c. Append n_step_return to returns
    # 3. Return returns list
    
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    
    return returns

In [ ]:
# Test your implementation
compute_n_step_returns_test(compute_n_step_returns)

## Exercise 2: Actor Network

The Actor network is similar to basic Actor-Critic, but we'll prepare it for better integration with advantages.

**Architecture**: Same as before - outputs action probabilities.

**Task**: Implement the Actor network.

In [ ]:
# GRADED FUNCTION: A2CActorNetwork

class A2CActorNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        """
        Actor network for A2C.
        
        Arguments:
        state_dim -- dimension of state space
        action_dim -- dimension of action space
        hidden_dim -- number of hidden units
        """
        super(A2CActorNetwork, self).__init__()
        
        # (approx. 2 lines)
        # Define two fully connected layers:
        # fc1: state_dim -> hidden_dim
        # fc2: hidden_dim -> action_dim
        
        # YOUR CODE STARTS HERE
        
        
        # YOUR CODE ENDS HERE
    
    def forward(self, state):
        """
        Forward pass to get action probabilities.
        
        Arguments:
        state -- state tensor
        
        Returns:
        action_probs -- probability distribution over actions
        """
        # (approx. 3 lines)
        # 1. Pass through fc1 with ReLU
        # 2. Pass through fc2
        # 3. Apply softmax for probabilities
        
        # YOUR CODE STARTS HERE
        
        
        # YOUR CODE ENDS HERE
        
        return action_probs

In [ ]:
# Test your implementation
a2c_actor_network_test(A2CActorNetwork)

## Exercise 3: Critic Network

The Critic estimates state values V(s). Same as basic Actor-Critic.

**Task**: Implement the Critic network.

In [ ]:
# GRADED FUNCTION: A2CCriticNetwork

class A2CCriticNetwork(nn.Module):
    def __init__(self, state_dim, hidden_dim=128):
        """
        Critic network for A2C.
        
        Arguments:
        state_dim -- dimension of state space
        hidden_dim -- number of hidden units
        """
        super(A2CCriticNetwork, self).__init__()
        
        # (approx. 2 lines)
        # Define two fully connected layers:
        # fc1: state_dim -> hidden_dim
        # fc2: hidden_dim -> 1 (value output)
        
        # YOUR CODE STARTS HERE
        
        
        # YOUR CODE ENDS HERE
    
    def forward(self, state):
        """
        Forward pass to get state value.
        
        Arguments:
        state -- state tensor
        
        Returns:
        value -- estimated value of the state
        """
        # (approx. 3 lines)
        # 1. Pass through fc1 with ReLU
        # 2. Pass through fc2
        # 3. Squeeze to scalar
        
        # YOUR CODE STARTS HERE
        
        
        # YOUR CODE ENDS HERE
        
        return value

In [ ]:
# Test your implementation
a2c_critic_network_test(A2CCriticNetwork)

## Exercise 4: Compute A2C Loss

A2C uses **n-step advantages** instead of 1-step TD error:

**Advantage**:
$$A(s_t, a_t) = R_t^{(n)} - V(s_t)$$

where $R_t^{(n)}$ is the n-step return we computed earlier.

**Actor Loss** (Policy Gradient):
$$L_{actor} = -\log \pi(a|s) \cdot A(s,a)$$

**Critic Loss** (MSE):
$$L_{critic} = [R_t^{(n)} - V(s_t)]^2 = A(s,a)^2$$

**Optional Entropy Bonus** (for exploration):
$$L_{entropy} = -\beta \sum_a \pi(a|s) \log \pi(a|s)$$

**Task**: Implement the A2C loss function.

In [ ]:
# GRADED FUNCTION: compute_a2c_loss

def compute_a2c_loss(log_probs, values, n_step_returns, entropy_coef=0.01):
    """
    Compute A2C loss with entropy bonus.
    
    Arguments:
    log_probs -- list of log probabilities of actions taken
    values -- list of state values V(s) from critic
    n_step_returns -- list of n-step returns R^(n)
    entropy_coef -- coefficient for entropy bonus (encourages exploration)
    
    Returns:
    actor_loss -- policy gradient loss
    critic_loss -- value function loss
    entropy_loss -- entropy bonus (for logging)
    total_loss -- combined loss for backprop
    """
    # (approx. 12-15 lines)
    # 1. Stack tensors:
    #    log_probs_tensor = torch.stack(log_probs)
    #    values_tensor = torch.stack(values)
    #    returns_tensor = torch.tensor(n_step_returns)
    # 2. Compute advantages:
    #    advantages = returns_tensor - values_tensor.detach()
    #    Note: detach advantages for actor (don't backprop through critic to actor)
    # 3. Actor loss:
    #    actor_loss = -(log_probs_tensor * advantages).mean()
    # 4. Critic loss:
    #    critic_loss = F.mse_loss(values_tensor, returns_tensor)
    # 5. Entropy (optional, for exploration):
    #    Can compute from log_probs or just set to 0 for simplicity
    # 6. Total loss:
    #    total_loss = actor_loss + critic_loss
    
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    
    return actor_loss, critic_loss, total_loss

In [ ]:
# Test your implementation
compute_a2c_loss_test(compute_a2c_loss)

## Exercise 5: Train A2C

Now let's put it all together! A2C training collects n-step trajectories and updates both networks.

**Algorithm** (per episode):
1. Collect trajectory of n steps (or until done)
2. Compute n-step returns for all states
3. Compute A2C loss
4. Update both actor and critic
5. Repeat from next state

**Key difference from basic Actor-Critic**: We update every n steps, not every step!

**Task**: Implement the A2C training loop.

In [ ]:
# GRADED FUNCTION: train_a2c

def train_a2c(env, actor, critic, optimizer, n_episodes=500, gamma=0.99, 
              n_steps=5, max_steps_per_episode=500):
    """
    Train A2C on the environment.
    
    Arguments:
    env -- Gym environment
    actor -- Actor network
    critic -- Critic network
    optimizer -- shared optimizer for both networks
    n_episodes -- number of episodes to train
    gamma -- discount factor
    n_steps -- number of steps for n-step returns
    max_steps_per_episode -- max steps per episode
    
    Returns:
    episode_rewards -- list of total rewards per episode
    """
    episode_rewards = []
    
    # (approx. 40-45 lines)
    # For each episode:
    #   1. Reset environment
    #   2. Initialize trajectory buffers:
    #      - states, actions, rewards, log_probs, values, dones
    #   3. total_reward = 0
    #   4. For each step:
    #      a. Get action from actor (with log_prob)
    #      b. Get value from critic
    #      c. Take action in environment
    #      d. Store experience in buffers
    #      e. If collected n_steps experiences OR done:
    #         - Get final bootstrap value (if not done)
    #         - Compute n-step returns
    #         - Compute A2C loss
    #         - Update networks:
    #           * optimizer.zero_grad()
    #           * total_loss.backward()
    #           * optimizer.step()
    #         - Clear buffers (but keep last state if not done)
    #      f. Update state and total_reward
    #      g. If done: break
    #   5. Append total_reward
    #   6. Print progress every 50 episodes
    
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    
    return episode_rewards

In [ ]:
# Test your implementation
train_a2c_test(train_a2c, A2CActorNetwork, A2CCriticNetwork)

## Full Training Run

Let's train A2C on CartPole!

In [ ]:
# Initialize networks
actor = A2CActorNetwork(state_dim, action_dim)
critic = A2CCriticNetwork(state_dim)

# Use shared optimizer (common practice in A2C)
optimizer = optim.Adam(list(actor.parameters()) + list(critic.parameters()), lr=1e-3)

# Train
episode_rewards = train_a2c(
    env, actor, critic, optimizer,
    n_episodes=500,
    gamma=0.99,
    n_steps=5
)

# Plot results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(episode_rewards, alpha=0.6)
plt.plot(np.convolve(episode_rewards, np.ones(50)/50, mode='valid'), linewidth=2)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('A2C Training Progress')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
window = 100
if len(episode_rewards) >= window:
    moving_avg = np.convolve(episode_rewards, np.ones(window)/window, mode='valid')
    plt.plot(moving_avg)
    plt.axhline(y=475, color='r', linestyle='--', label='Solved (475)')
    plt.xlabel('Episode')
    plt.ylabel(f'Average Reward (last {window})')
    plt.title('Moving Average')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

if len(episode_rewards) >= 100:
    final_avg = np.mean(episode_rewards[-100:])
    if final_avg >= 475:
        print(f"\n🎉 Environment solved! Final average: {final_avg:.2f}")
    else:
        print(f"\n📊 Training completed. Final average: {final_avg:.2f}")

## Comparison: Actor-Critic vs A2C

### Why A2C is Better

**1. Bias-Variance Tradeoff**:
- **1-step TD** (Actor-Critic): Lower variance, higher bias
- **Monte Carlo** (REINFORCE): Higher variance, no bias
- **n-step TD** (A2C): **Best of both worlds!**

**2. Credit Assignment**:
- A2C looks n steps ahead, better at assigning credit for actions
- Especially useful in environments with delayed rewards

**3. Sample Efficiency**:
- More informative updates lead to faster learning

### When to Use Each

- **Basic Actor-Critic**: Simple environments, online learning priority
- **A2C**: Most practical applications (better performance)
- **A3C**: Parallel training with multiple workers (asynchronous)

### Hyperparameter: n_steps

- **Small n (2-5)**: Lower variance, higher bias, faster updates
- **Large n (20-50)**: Higher variance, lower bias, closer to Monte Carlo
- **Typical**: n=5 is a good default for many environments

## A3C: Asynchronous Version

**A3C** is A2C with multiple parallel workers:
- Each worker runs independently with its own environment
- Workers asynchronously update shared network parameters
- Better exploration due to diverse experiences
- Faster training with parallelization

**A2C vs A3C**:
- A2C is synchronous (easier to implement, more stable)
- A3C is asynchronous (faster with multi-core CPUs)
- Modern practice often uses A2C with vectorized environments

## Congratulations!

You've successfully implemented A2C! You now understand:
- ✅ N-step returns and why they improve learning
- ✅ The bias-variance tradeoff in RL
- ✅ How to compute proper advantages
- ✅ Batch updates with trajectory collection
- ✅ The relationship between A2C and A3C

**Next Steps**:
- Try **PPO** (Proximal Policy Optimization) for even more stable training
- Experiment with different n_steps values
- Implement **A3C** with multiprocessing for parallel training